In [24]:
import pyximport; pyximport.install()
from utils import *
import time
import pandas as pd

In [25]:
from os import listdir
instances=listdir('Instances/')
instances=[x for x in [(x.split('.'))[0] for x in instances] if len(x) ==2]

In [33]:
instances

['G2',
 'K1',
 'J4',
 'G3',
 'G1',
 'K2',
 'K3',
 'F4',
 'G4',
 'J3',
 'J2',
 'G5',
 'F3',
 'K4',
 'J1',
 'F2',
 'G6',
 'A1',
 'M2',
 'M3',
 'A2',
 'L5',
 'M1',
 'L4',
 'A3',
 'M4',
 'L1',
 'A4',
 'L3',
 'L2',
 'N6',
 'C1',
 'C3',
 'N4',
 'N5',
 'C2',
 'B2',
 'N1',
 'B3',
 'B1',
 'N2',
 'N3',
 'C4',
 'E3',
 'H4',
 'H5',
 'I1',
 'E2',
 'D4',
 'I3',
 'I2',
 'H6',
 'E1',
 'D1',
 'H2',
 'H3',
 'D2',
 'I5',
 'H1',
 'I4',
 'D3']

In [27]:
%%time
record ={}
list_cost =[]

start = time.time()
for e in instances:
    folder_instances = 'Instances/'
    folder_rpa_solutions = 'RPA_Solutions/'
    path = folder_instances+e+'.txt'
    path_solutions =folder_rpa_solutions+'Detailed_Solution_'+e+'.txt'

    start = time.time()
    count, number_of_customers, one, number_of_vehicles,\
    vect_cord, vect_domanda, vect_carico, vec_b, vec_l, capacity = parser_instance(path)  

    distance = distance_matrix(vect_cord)
    saving=saving_matrix(distance)
    
    dict_Saving_Order=savingOrder(saving)
    
    dict_b, dict_l , dict_merge= route_Linehaul_Backhaul(dict_Saving_Order, vec_b, vec_l)
    
    
    list_vehicle, set_visited = init_list_of_vehicle(dict_l, vect_domanda, distance, number_of_vehicles)
    route_linehaul(dict_l, list_vehicle, vect_domanda, set_visited, distance,  capacity) #funzione void
    route_backhaul(list_vehicle, dict_merge, dict_b, set_visited, vect_carico, distance, number_of_customers, capacity) #funzione void
    tot_cost=sum([x.cost for x in list_vehicle])
    record[e]=round(((tot_cost * 100) / extract_total_cost_BP(path_solutions))-100 , 2)
    list_cost.append([e,tot_cost])

duration = time.time()-start

CPU times: user 12.1 s, sys: 67.6 ms, total: 12.2 s
Wall time: 12.3 s


In [30]:
df_cost = pd.DataFrame(list_cost, columns = ['Instances', 'Best_Solution']) 

In [32]:
df_cost.to_csv('Risultati/costo_serialSolution.csv')

In [5]:
tot_errore=0
for e in record.keys():
    tot_errore += record[e]

tot_errore/len(record)

61.51081967213113

In [6]:
df_data = pd.DataFrame({'': [number_of_customers, capacity, number_of_vehicles,
                            len(set_visited)-1, tot_cost, extract_total_cost_BP(path_solutions),
                            round(((tot_cost * 100) / extract_total_cost_BP(path_solutions))-100 , 2), duration]},
                       index=['Customers', 'Max Load', 'Routes Of the Solution', 'nodes visited',
                              'Total Cost', 'Total Cost BP', 'Error', 'Time for Instance'
                             ])

In [7]:
df_data

,
Customers,38.000000
Max Load,2750.000000
Routes Of the Solution,7.000000
nodes visited,38.000000
Total Cost,363507.496288
Total Cost BP,239479.000000
Error,51.790000
Time for Instance,0.009944


In [8]:
#support
count = 0
best_route = []
flag = False

with open(path_solutions) as file:
    for line in file:
        if "ROUTE" in line:
            flag = True
        if flag:
            count+=1
        if count == 7:
            best_route.append([int(x) for x in line.split()])
            flag=False
            count=0


In [9]:
best_route

[[0, 33, 25, 38, 35, 19, 0],
 [0, 30, 14, 21, 37, 15, 0],
 [0, 29, 34, 18, 7, 0],
 [0, 10, 28, 16, 9, 11, 2, 0],
 [0, 12, 27, 17, 1, 6, 3, 0],
 [0, 31, 23, 13, 24, 8, 5, 4, 0],
 [0, 20, 26, 32, 22, 36, 0]]

In [10]:
my_route=[x.route for x in list_vehicle]
my_route

[[0, 13, 24, 36, 22, 20, 1, 6, 3, 8, 5, 0],
 [0, 17, 27, 12, 37, 14, 33, 2, 7, 4, 0],
 [0, 26, 32, 18, 38, 28, 0],
 [0, 23, 31, 25, 10, 0],
 [0, 19, 35, 16, 30, 0],
 [0, 9, 11, 21, 15, 0],
 [0, 29, 34, 0]]

In [11]:
best_cost=0
for element in best_route:
    for i,v in enumerate(element):
        if i > 0:
            best_cost+=distance[element[i-1]][element[i]]

In [12]:
my_cost=0
for element in my_route:
    for i,v in enumerate(element):
        if i > 0:
            my_cost+=distance[element[i-1]][element[i]]

In [13]:
best_cost

239478.6325468138

In [14]:
my_cost

363507.4962878069

In [15]:
round((my_cost * 100) / best_cost -100 , 2)

51.79

In [17]:
list_record=[]
for e in record.keys():
    list_record.append([e,record[e]])

In [18]:
df = pd.DataFrame(list_record, columns = ['Instances', 'Error']) 

In [19]:
df2 = pd.DataFrame([['Total Average', tot_errore/len(record)]], columns = ['Instances', 'Error']) 

In [20]:
df.append(df2)

,Instances,Error
0,G2,60.33000
1,K1,61.98000
2,J4,62.88000
3,G3,67.50000
4,G1,73.21000
...,...,...
57,I5,72.91000
58,H1,48.75000
59,I4,68.47000
60,D3,51.79000


In [21]:
df.to_csv('Risultati/seriale_completo.csv')